In [ ]:
!pip install torch_geometric
!pip install grakel


In [21]:
# Autoencoder

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import GINConv
from torch_geometric.nn import global_add_pool


# Decoder
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, n_layers, n_nodes):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.n_nodes = n_nodes

        mlp_layers = (
            [nn.Linear(latent_dim, hidden_dim)] +
            [nn.Linear(hidden_dim, hidden_dim) for i in range(n_layers - 2)]
            )
        mlp_layers.append(nn.Linear(hidden_dim, 2*n_nodes*(n_nodes-1)//2))

        self.mlp = nn.ModuleList(mlp_layers)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, data):
        x=data.x
        for i in range(self.n_layers-1):
            x = self.relu(self.mlp[i](x))

        x = self.mlp[self.n_layers-1](x)
        x = torch.reshape(x, (x.size(0), -1, 2))
        x = F.gumbel_softmax(x, tau=1, hard=True)[:, :, 0]

        adj = torch.zeros(x.size(0),
                          self.n_nodes,
                          self.n_nodes,
                          device=x.device)

        idx = torch.triu_indices(self.n_nodes, self.n_nodes, 1)
        adj[:, idx[0], idx[1]] = x
        adj = adj + torch.transpose(adj, 1, 2)
        return adj
class NewDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, n_layers, n_nodes, cond_dim=7):

        super(NewDecoder, self).__init__()
        self.n_layers = n_layers
        self.n_nodes = n_nodes
        self.cond_dim = cond_dim

        input_dim = latent_dim + cond_dim if cond_dim else latent_dim

        mlp_layers = (
            [nn.Linear(input_dim, hidden_dim)] +
            [nn.Linear(hidden_dim, hidden_dim) for i in range(n_layers - 2)]
        )
        mlp_layers.append(nn.Linear(hidden_dim, 2 * n_nodes * (n_nodes - 1) // 2))

        self.mlp = nn.ModuleList(mlp_layers)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, cond):


        for i in range(cond.size(1)):  # Iterate over features
            col_mean = torch.nanmean(cond[:, i])  # Mean for each feature
            cond[:, i] = torch.nan_to_num(cond[:, i], nan=col_mean)
        min_val = torch.min(cond, dim=0)[0]
        max_val = torch.max(cond, dim=0)[0]
        cond = (cond - min_val) / (max_val - min_val)



        x = torch.cat([x, cond], dim=-1)

        # Pass through MLP
        for i in range(self.n_layers - 1):
            x = self.relu(self.mlp[i](x))
        x = self.mlp[self.n_layers - 1](x)

        # Reshape and apply Gumbel-Softmax
        x = torch.reshape(x, (x.size(0), -1, 2))
        x = F.gumbel_softmax(x, tau=1, hard=True)[:, :, 0]

        # Create adjacency matrix
        adj = torch.zeros(x.size(0), self.n_nodes, self.n_nodes, device=x.device)
        idx = torch.triu_indices(self.n_nodes, self.n_nodes, 1)
        adj[:, idx[0], idx[1]] = x
        adj = adj + torch.transpose(adj, 1, 2)
        return adj





class GIN(torch.nn.Module):
    def __init__(self,
                 input_dim,
                 hidden_dim,
                 latent_dim,
                 n_layers,
                 n_max_nodes,
                 dropout=0.4,
                 cond_dim=7,
                 d_cond=16):
        super().__init__()
        self.dropout = dropout
        self.mlp_adj = nn.Sequential(
            nn.Linear(n_max_nodes ** 2, latent_dim),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(latent_dim),
            nn.Linear(latent_dim, latent_dim),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(latent_dim),
            nn.Linear(latent_dim, input_dim),
            nn.LeakyReLU(0.1)
        )

        self.convs = torch.nn.ModuleList()
        self.convs.append(GINConv(
                            nn.Sequential(
                                nn.Linear(input_dim,
                                          hidden_dim),
                                nn.LeakyReLU(0.2),
                                nn.BatchNorm1d(hidden_dim),
                                nn.Linear(hidden_dim, hidden_dim),
                                nn.LeakyReLU(0.2))
                            ))
        for layer in range(n_layers-1):
            self.convs.append(GINConv(
                                nn.Sequential(
                                    nn.Linear(hidden_dim,
                                              hidden_dim),
                                    nn.LeakyReLU(0.2),
                                    nn.BatchNorm1d(hidden_dim),
                                    nn.Linear(hidden_dim, hidden_dim),
                                    nn.LeakyReLU(0.2))
                                    ))
        self.cond_mlp = nn.Sequential(
            nn.Linear(cond_dim, d_cond),
            nn.ReLU(),
            nn.Linear(d_cond, d_cond),
        )

        self.bn = nn.BatchNorm1d(hidden_dim)
        self.fc = nn.Linear(hidden_dim, latent_dim)
        self.concat_conv = nn.Sequential(
            nn.Linear(hidden_dim + cond_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(hidden_dim))

    def forward(self, data):
        edge_index = data.edge_index
        x = data.x
        #A = data.A
        cond=data.stats
        for i in range(cond.size(1)):  # Iterate over features
            col_mean = torch.nanmean(cond[:, i])  # Mean for each feature
            cond[:, i] = torch.nan_to_num(cond[:, i], nan=col_mean)
        min_val = torch.min(cond, dim=0)[0]
        max_val = torch.max(cond, dim=0)[0]
        cond = (cond - min_val) / (max_val - min_val)
        #cond=self.cond_mlp(cond)
        # Flatten symmetric matrix A to vector
        #A = A.view(A.size(0), -1)
        #vec_A = self.mlp_adj(A)
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.dropout(x, self.dropout, training=self.training)

        x = global_add_pool(x, data.batch)
        #x = torch.cat((x, vec_A), dim=1)
        x = torch.cat((x, cond), dim=1)

        x = self.concat_conv(x)
        out = self.bn(x)
        out = self.fc(x)
        return out

# Variational Autoencoder
class VariationalAutoEncoder(nn.Module):
    def __init__(self,
                 input_dim,
                 hidden_dim_enc,
                 hidden_dim_dec,
                 latent_dim,
                 n_layers_enc,
                 n_layers_dec,
                 n_max_nodes):
        super(VariationalAutoEncoder, self).__init__()
        self.n_max_nodes = n_max_nodes
        self.input_dim = input_dim
        self.encoder = GIN(input_dim,
                           hidden_dim_enc,
                           hidden_dim_enc,
                           n_layers_enc,
                           n_max_nodes)
        self.fc_mu = nn.Linear(hidden_dim_enc, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim_enc, latent_dim)
        self.decoder = NewDecoder(latent_dim,
                               hidden_dim_dec,
                               n_layers_dec,
                               n_max_nodes)

    def forward(self, data):
        x_g = self.encoder(data)
        mu = self.fc_mu(x_g)
        logvar = self.fc_logvar(x_g)
        x_g = self.reparameterize(mu, logvar)
        adj = self.decoder(x_g,data.stats)
        return adj

    def encode(self, data):
        x_g = self.encoder(data)
        mu = self.fc_mu(x_g)
        logvar = self.fc_logvar(x_g)
        x_g = self.reparameterize(mu, logvar)
        return x_g

    def reparameterize(self, mu, logvar, eps_scale=1.):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.randn_like(std) * eps_scale
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, mu, logvar,cond):
        x_g = self.reparameterize(mu, logvar)
        adj = self.decoder(x_g,cond)
        return adj

    def decode_mu(self, mu, cond,threshold=0.5):
        adj_logits = self.decoder(mu,cond)
        adj_probs = torch.sigmoid(adj_logits)
        adj = adj_probs * (1 - torch.eye(adj_probs.size(-2), adj_probs.size(-1), device=adj_probs.device))
        # Thresholding to get binary adjacency
        adj_binary = (adj > threshold).float()
        return adj_binary

    def loss_function(self, data, beta=0.01):
        x_g = self.encoder(data)
        mu = self.fc_mu(x_g)
        logvar = self.fc_logvar(x_g)
        x_g = self.reparameterize(mu, logvar)
        adj = self.decoder(x_g,data.stats)
        #recon = F.binary_cross_entropy_with_logits(adj, data.A, reduction='sum')
        recon =  F.l1_loss(adj, data.A, reduction='sum')
        kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        loss = recon + beta*kld

        return loss, recon, kld


In [14]:
# Denoise model

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

def extract(a, t, x_shape):
    """
    Extract values from a tensor `a` at positions defined by `t`.

    Parameters:
        a (torch.Tensor): The tensor to extract from.
        t (torch.Tensor): Indices tensor.
        x_shape (tuple): Target shape for the output.

    Returns:
        torch.Tensor: Extracted values reshaped to match `x_shape`.
    """
    batch_size = t.shape[0]
    out = a.gather(-1, t.cpu())
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)


def q_sample(x_start, t, sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod,
             noise=None):
    """
    Forward diffusion using a predefined property.

    Parameters:
        x_start (torch.Tensor): Initial input tensor.
        t (torch.Tensor): Time step tensor.
        sqrt_alphas_cumprod (torch.Tensor): Cumulative product of alphas.
        sqrt_one_minus_alphas_cumprod (torch.Tensor):
        C
umulative product for noise.
        noise (torch.Tensor, optional): Random noise tensor. Defaults to None.

    Returns:
        torch.Tensor: Noisy input after forward diffusion.
    """
    if noise is None:
        noise = torch.randn_like(x_start)

    sqrt_alphas_cumprod_t = extract(sqrt_alphas_cumprod, t, x_start.shape)
    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t, x_start.shape
    )

    output = (sqrt_alphas_cumprod_t * x_start
              + sqrt_one_minus_alphas_cumprod_t * noise)

    return output


def p_losses(denoise_model, x_start, t, cond, sqrt_alphas_cumprod,
             sqrt_one_minus_alphas_cumprod, noise=None, loss_type="l1"):
    """
    Calculate the loss for denoising.

    Parameters:
        denoise_model (nn.Module): The denoising model.
        x_start (torch.Tensor): Original data.
        t (torch.Tensor): Time steps.
        cond (torch.Tensor): Conditioning information.
        sqrt_alphas_cumprod (torch.Tensor): Cumulative product of alphas.
        sqrt_one_minus_alphas_cumprod (torch.Tensor): Noise term.
        noise (torch.Tensor, optional): Random noise. Defaults to None.
        loss_type (str): Type of loss. Can be "l1", "l2", or "huber".

    Returns:
        torch.Tensor: Computed loss.
    """
    if noise is None:
        noise = torch.randn_like(x_start)

    x_noisy = q_sample(x_start, t, sqrt_alphas_cumprod,
                       sqrt_one_minus_alphas_cumprod, noise=noise)
    predicted_noise = denoise_model(x_noisy, t, cond)

    if loss_type == 'l1':
        loss = F.l1_loss(noise, predicted_noise)
    elif loss_type == 'l2':
        loss = F.mse_loss(noise, predicted_noise)
    elif loss_type == "huber":
        loss = F.smooth_l1_loss(noise, predicted_noise)
    else:
        raise NotImplementedError(f"Loss type {loss_type} is not implemented.")

    return loss


class SinusoidalPositionEmbeddings(nn.Module):
    """
    Module for generating sinusoidal position embeddings.
    """
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.arange(half_dim, device=device) * -embeddings
        embeddings = torch.exp(embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings


class DenoiseNN(nn.Module):
    """
    Denoising neural network with temporal and conditional embeddings.
    """
    def __init__(self, input_dim, hidden_dim, n_layers, n_cond, d_cond):
        super().__init__()
        self.n_layers = n_layers
        self.n_cond = n_cond


        # Conditional embedding network
        self.cond_mlp = nn.Sequential(
            nn.Linear(self.n_cond, d_cond),
            nn.ReLU(),
            nn.Linear(d_cond, d_cond),
        )

        # Time embedding network
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, hidden_dim),
        )

        # Main network
        mlp_layers = (
            [nn.Linear(input_dim + d_cond, hidden_dim)] +
            [nn.Linear(hidden_dim + d_cond, hidden_dim)
             for _ in range(n_layers - 2)]
        )
        mlp_layers.append(nn.Linear(hidden_dim, input_dim))
        self.mlp = nn.ModuleList(mlp_layers)

        # Batch normalization
        self.bn = nn.ModuleList(
            [nn.BatchNorm1d(hidden_dim) for _ in range(n_layers - 1)]
        )

        self.relu = nn.ReLU()

    def forward(self, x, t, cond):

        cond = cond.view(-1, self.n_cond)
        for i in range(cond.size(1)):  # Iterate over features
            col_mean = torch.nanmean(cond[:, i])  # Mean for each feature
            cond[:, i] = torch.nan_to_num(cond[:, i], nan=col_mean)
        min_val = torch.min(cond, dim=0)[0]
        max_val = torch.max(cond, dim=0)[0]
        cond = (cond - min_val) / (max_val - min_val)



        cond = self.cond_mlp(cond)
        t = self.time_mlp(t)

        for i in range(self.n_layers - 1):
            x = torch.cat((x, cond), dim=1)
            x = self.relu(self.mlp[i](x)) + t
            x = self.bn[i](x)

        x = self.mlp[self.n_layers - 1](x)
        return x


@torch.no_grad()
def p_sample(model, x, t, cond, t_index, betas):
    """
    Perform a single reverse diffusion step.

    Parameters:
        model (nn.Module): The denoising model.
        x (torch.Tensor): Noisy input.
        t (torch.Tensor): Time step tensor.
        cond (torch.Tensor): Conditional input.
        t_index (int): Current timestep index.
        betas (torch.Tensor): Beta schedule.

    Returns:
        torch.Tensor: Output tensor after one reverse step.
    """
    alphas = 1.0 - betas
    alphas_cumprod = torch.cumprod(alphas, axis=0)
    alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
    sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
    # sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
    sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)

    posterior_variance = (
        betas * (1.0 - alphas_cumprod_prev) / (1.0 - alphas_cumprod)
    )

    betas_t = extract(betas, t, x.shape)
    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t, x.shape
    )
    sqrt_recip_alphas_t = extract(sqrt_recip_alphas, t, x.shape)

    model_mean = sqrt_recip_alphas_t * (
        x - betas_t * model(x, t, cond) / sqrt_one_minus_alphas_cumprod_t
    )

    if t_index == 0:
        return model_mean

    posterior_variance_t = extract(posterior_variance, t, x.shape)
    noise = torch.randn_like(x)
    return model_mean + torch.sqrt(posterior_variance_t) * noise


@torch.no_grad()
def p_sample_loop(model, cond, timesteps, betas, shape):
    """
    Perform a full sampling loop over all timesteps.

    Parameters:
        model (nn.Module): Denoising model.
        cond (torch.Tensor): Conditional input.
        timesteps (int): Number of timesteps.
        betas (torch.Tensor): Beta schedule.
        shape (tuple): Shape of the output tensor.

    Returns:
        list[torch.Tensor]: List of sampled images at each timestep.
    """
    device = next(model.parameters()).device
    img = torch.randn(shape, device=device)
    imgs = []

    for i in reversed(range(timesteps)):
        img = p_sample(
            model,
            img,
            torch.full((shape[0],), i, device=device, dtype=torch.long),
            cond,
            i,
            betas
        )
        imgs.append(img)

    return imgs


@torch.no_grad()
def sample(model, cond, latent_dim, timesteps, betas, batch_size):
    """
    Sample images from the model.

    Parameters:
        model (nn.Module): Denoising model.
        cond (torch.Tensor): Conditional input.
        latent_dim (int): Dimensionality of the latent space.
        timesteps (int): Number of timesteps.
        betas (torch.Tensor): Beta schedule.
        batch_size (int): Number of samples in the batch.

    Returns:
        list[torch.Tensor]: Generated samples.
    """
    return p_sample_loop(model, cond, timesteps, betas, shape=(batch_size,
                                                               latent_dim))


In [4]:
# Extracts feats

import os
from tqdm import tqdm
import random
import re

random.seed(32)


def extract_numbers(text):
    # Use regular expression to find integers and floats
    numbers = re.findall(r'\d+\.\d+|\d+', text)
    # Convert the extracted numbers to float
    return [float(num) for num in numbers]


def extract_feats(file):
    stats = []
    fread = open(file, "r")
    line = fread.read()
    line = line.strip()
    stats = extract_numbers(line)
    fread.close()
    return stats


In [5]:
# Utils

import os
import math
import requests
import zipfile
import networkx as nx
import numpy as np
import scipy as sp
import scipy.sparse
import torch
import torch.nn.functional as F
import community as community_louvain

from torch import Tensor
from torch.utils.data import Dataset

from grakel.utils import graph_from_networkx
from grakel.kernels import WeisfeilerLehman, VertexHistogram
from tqdm import tqdm
import scipy.sparse as sparse
from torch_geometric.data import Data


import requests
import zipfile
import os
import shutil

# Extracts feats

import os
from tqdm import tqdm
import random
import re

random.seed(32)


def extract_numbers(text):
    # Use regular expression to find integers and floats
    numbers = re.findall(r'\d+\.\d+|\d+', text)
    # Convert the extracted numbers to float
    return [float(num) for num in numbers]


def extract_feats(file):
    stats = []
    fread = open(file, "r")
    line = fread.read()
    line = line.strip()
    stats = extract_numbers(line)
    fread.close()
    return stats


def load_data(repo_url, download_dir):
    try:
        zip_url = f"{repo_url}/archive/refs/heads/main.zip"  # Update branch if not "main"
        response = requests.get(zip_url)
        response.raise_for_status()

        # Save zip file
        zip_path = os.path.join(download_dir, "repo.zip")
        with open(zip_path, "wb") as f:
            f.write(response.content)

        # Extract zip file
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(download_dir)

        # Clean up zip file
        os.remove(zip_path)

        # Locate the folder that was created after extracting the repo
        repo_folder_name = os.path.basename(repo_url) + "-main"
        repo_folder_path = os.path.join(download_dir, repo_folder_name)

        # Path to the new DATA folder
        data_folder_path = os.path.join(download_dir, "data")
        os.makedirs(data_folder_path, exist_ok=True)

        # Path to the data.zip inside the extracted repo folder
        data_zip_path = os.path.join(repo_folder_path, "data.zip")

        # Extract data.zip into the DATA folder
        with zipfile.ZipFile(data_zip_path, "r") as zip_ref:
            zip_ref.extractall(data_folder_path)

        # Remove the repo folder after extracting data.zip
        shutil.rmtree(repo_folder_path)

        print(f"Repository and data extracted successfully. data folder created at {data_folder_path}")
    except Exception as e:
        print(f"Error: {e}")


def preprocess_dataset(dataset, n_max_nodes, spectral_emb_dim):
    data_lst = []

    if dataset == 'test':
        desc_file = './data/data/' + dataset + '/test.txt'
        fr = open(desc_file, "r")
        for line in fr:
            line = line.strip()
            tokens = line.split(",")
            graph_id = tokens[0]
            desc = tokens[1:]
            desc = "".join(desc)
            feats_stats = extract_numbers(desc)
            feats_stats = torch.FloatTensor(feats_stats).unsqueeze(0)
            data_lst.append(Data(stats=feats_stats, filename=graph_id))
        fr.close()

    else:
        graph_path = './data/data/' + dataset + '/graph'
        desc_path = './data/data/' + dataset + '/description'

        # Traverse through all the graphs in the folder
        files = [f for f in os.listdir(graph_path)]
        for fileread in tqdm(files):
            tokens = fileread.split("/")
            idx = tokens[-1].find(".")
            filen = tokens[-1][:idx]
            extension = tokens[-1][idx + 1:]
            fread = os.path.join(graph_path, fileread)
            fstats = os.path.join(desc_path, filen + ".txt")

            # Load dataset to networkx
            if extension == "graphml":
                G = nx.read_graphml(fread)
                G = nx.convert_node_labels_to_integers(G, ordering="sorted")
            else:
                G = nx.read_edgelist(fread)

            # Use BFS for adjacency matrix creation
            CGs = [G.subgraph(c) for c in nx.connected_components(G)]
            CGs = sorted(CGs, key=lambda x: x.number_of_nodes(), reverse=True)

            node_list_bfs = []
            for ii in range(len(CGs)):
                node_degree_list = [(n, d) for n, d in CGs[ii].degree()]
                degree_sequence = sorted(node_degree_list, key=lambda tt: tt[1], reverse=True)
                bfs_tree = nx.bfs_tree(CGs[ii], source=degree_sequence[0][0])
                node_list_bfs += list(bfs_tree.nodes())

            adj_bfs = nx.to_numpy_array(G, nodelist=node_list_bfs)
            adj = torch.from_numpy(adj_bfs).float()
            diags = np.sum(adj_bfs, axis=0)
            diags = np.squeeze(np.asarray(diags))
            D = sparse.diags(diags).toarray()
            L = D - adj_bfs
            with np.errstate(divide="ignore"):
                diags_sqrt = 1.0 / np.sqrt(diags)
            diags_sqrt[np.isinf(diags_sqrt)] = 0
            DH = sparse.diags(diags).toarray()
            L = np.linalg.multi_dot((DH, L, DH))
            L = torch.from_numpy(L).float()
            eigval, eigvecs = torch.linalg.eigh(L)
            eigval = torch.real(eigval)
            eigvecs = torch.real(eigvecs)
            idx = torch.argsort(eigval)
            eigvecs = eigvecs[:, idx]

            edge_index = torch.nonzero(adj).t()

            size_diff = n_max_nodes - G.number_of_nodes()
            x = torch.zeros(G.number_of_nodes(), spectral_emb_dim + 1)
            x[:, 0] = torch.mm(adj, torch.ones(G.number_of_nodes(), 1))[:, 0] / (n_max_nodes - 1)
            mn = min(G.number_of_nodes(), spectral_emb_dim)
            mn += 1
            x[:, 1:mn] = eigvecs[:, :spectral_emb_dim]
            adj = F.pad(adj, [0, size_diff, 0, size_diff])
            adj = adj.unsqueeze(0)

            feats_stats = extract_feats(fstats)
            feats_stats = torch.FloatTensor(feats_stats).unsqueeze(0)

            data_lst.append(Data(x=x, edge_index=edge_index, A=adj, stats=feats_stats, filename=filen))

    return data_lst

def construct_nx_from_adj(adj):
    G = nx.from_numpy_array(adj, create_using=nx.Graph)
    to_remove = []
    for node in G.nodes():
        if G.degree(node) == 0:
            to_remove.append(node)
    G.remove_nodes_from(to_remove)
    return G


def handle_nan(x):
    if math.isnan(x):
        return float(-100)
    return x


def masked_instance_norm2D(x: torch.Tensor,
                           mask: torch.Tensor,
                           eps: float = 1e-5):
    """
    x: [batch_size (N), num_objects (L), num_objects (L), features(C)]
    mask: [batch_size (N), num_objects (L), num_objects (L), 1]
    """
    mask = mask.view(x.size(0), x.size(1), x.size(2), 1).expand_as(x)
    # (N,C)
    mean = (torch.sum(x * mask, dim=[1, 2]) / torch.sum(mask, dim=[1, 2]))
    # (N,L,L,C)
    var_term = ((x - mean.unsqueeze(1).unsqueeze(1).expand_as(x)) * mask)**2
    # (N,C)
    var = (torch.sum(var_term, dim=[1, 2]) / torch.sum(mask, dim=[1, 2]))
    mean = mean.unsqueeze(1).unsqueeze(1).expand_as(x)  # (N, L, L, C)
    var = var.unsqueeze(1).unsqueeze(1).expand_as(x)    # (N, L, L, C)
    instance_norm = (x - mean) / torch.sqrt(var + eps)   # (N, L, L, C)
    instance_norm = instance_norm * mask
    return instance_norm


def masked_layer_norm2D(x: torch.Tensor,
                        mask: torch.Tensor,
                        eps: float = 1e-5):
    """
    x: [batch_size (N), num_objects (L), num_objects (L), features(C)]
    mask: [batch_size (N), num_objects (L), num_objects (L), 1]
    """
    mask = mask.view(x.size(0), x.size(1), x.size(2), 1).expand_as(x)
    # (N)
    mean = torch.sum(x * mask, dim=[3, 2, 1]) / torch.sum(mask, dim=[3, 2, 1])
    # (N,L,L,C)
    var_term = ((x - mean.view(-1, 1, 1, 1).expand_as(x)) * mask)**2
    var = (torch.sum(var_term, dim=[3, 2, 1]) / torch.sum(mask, dim=[3, 2, 1]))
    # (N)
    mean = mean.view(-1, 1, 1, 1).expand_as(x)  # (N, L, L, C)
    var = var.view(-1, 1, 1, 1).expand_as(x)    # (N, L, L, C)
    layer_norm = (x - mean) / torch.sqrt(var + eps)   # (N, L, L, C)
    layer_norm = layer_norm * mask
    return layer_norm


def cosine_beta_schedule(timesteps, s=0.008):
    """
    cosine schedule as proposed in https://arxiv.org/abs/2102.09672
    """
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s)
                               * torch.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 0.0001, 0.9999)


def linear_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start, beta_end, timesteps)


def quadratic_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start**0.5, beta_end**0.5, timesteps) ** 2


def sigmoid_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    betas = torch.linspace(-6, 6, timesteps)
    return torch.sigmoid(betas) * (beta_end - beta_start) + beta_start


def graph_features(adj_matrix):
    """
    Compute features of a graph from the adjacency matrix, removing trailing zero blocks.

    Parameters:
        adj_matrix (numpy.ndarray): Adjacency matrix of the graph.

    Returns:
        list: List of graph features.
    """
    G= construct_nx_from_adj(adj_matrix)

    # Compute graph features
    num_nodes = G.number_of_nodes()  # Number of nodes
    num_edges = G.number_of_edges()  # Number of edges
    avg_degree = sum(dict(G.degree()).values()) / num_nodes if num_nodes > 0 else 0  # Average degree
    has_triangle = any(len(cycle) == 3 for cycle in nx.cycle_basis(G))  # Check for triangles
    global_clustering = nx.transitivity(G)  # Global clustering coefficient
    k_core = max(nx.core_number(G).values()) if num_nodes > 0 else 0  # Maximum k-core value
    communities = list(nx.connected_components(G))  # Connected components as communities
    num_communities = len(communities)  # Number of communities
    triangle_counts = nx.triangles(G)  # Count of triangles for each node
    num_triangles = sum(triangle_counts.values()) // 3  # Total triangles (each counted 3 times)

    # Return the features as a list
    return [num_nodes, num_edges, avg_degree, num_triangles, global_clustering, k_core, num_communities]

def fast_graph_features(data):
    """
    Compute fast-to-compute features of a graph from the given Data object.

    Parameters:
        data (Data): PyTorch Geometric Data object with attributes:
            - edge_index: Edge list as a tensor.
            - x: Node features (optional).

    Returns:
        torch.Tensor: Tensor of fast-to-compute graph features.
    """
    edge_index = data.edge_index  # Edge list
    num_nodes = data.num_nodes

    # Compute number of edges
    num_edges = edge_index.size(1)

    # Compute degree and average degree
    degrees = torch.bincount(edge_index[0], minlength=num_nodes).float()  # Convert to float
    avg_degree = degrees.mean()

    # Prepare the feature vector
    features = torch.tensor(
        [
            num_nodes,
            num_edges,
            avg_degree.item(),
        ],
        device=edge_index.device,
        dtype=torch.float,
    )

    return features



In [6]:
# Import necessary libraries

import argparse
import os
import random
import scipy as sp
import pickle

import shutil
import csv
import ast

import scipy.sparse as sparse
from tqdm import tqdm
from torch import Tensor
import networkx as nx
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
from torch_geometric.data import Data

import torch.nn.functional as F
from torch_geometric.loader import DataLoader

from torch.utils.data import Subset
from sklearn.preprocessing import MinMaxScaler
np.random.seed(13)

In [ ]:
# Loading Data
# Here, we load the data from the repository.

repo_url = "https://github.com/chris-mrn/Data_Altegrad"
download_dir = os.getcwd()

# Load the data
load_data(repo_url, download_dir)
print("Data loaded!")

In [15]:
from types import SimpleNamespace

# Cell 3: Set up argument parser for model configuration
# This block sets up the arguments for configuring the model using SimpleNamespace.
args = SimpleNamespace(**{
    'lr': 1e-3,  # Learning rate for the optimizer (default: 0.001)
    'dropout': 0.0,  # Dropout rate (default: 0.0)
    'batch_size': 256,  # Batch size for training (default: 256)
    'epochs_autoencoder': 600,  # Training epochs for autoencoder (default: 200)
    'hidden_dim_encoder': 64,  # Hidden dimension size for encoder (default: 64)
    'hidden_dim_decoder': 512,  # Hidden dimension size for decoder (default: 256)
    'latent_dim': 32,  # Latent space dimensionality (default: 32)
    'n_max_nodes': 50,  # Maximum number of nodes (default: 50)
    'n_layers_encoder': 2,  # Number of encoder layers (default: 2)
    'n_layers_decoder': 3,  # Number of decoder layers (default: 3)
    'spectral_emb_dim': 10,  # Dimensionality of spectral embeddings (default: 10)
    'epochs_denoise': 300,  # Training epochs for denoising model (default: 100)
    'timesteps': 500,  # Number of timesteps for diffusion (default: 500)
    'hidden_dim_denoise': 512,  # Hidden dimension size for denoising model (default: 512)
    'n_layers_denoise': 3,  # Number of denoising model layers (default: 3)
    'train_autoencoder': True,  # Flag to toggle autoencoder training (default: enabled)
    'train_denoiser': True,  # Flag to toggle denoiser training (default: enabled)
    'dim_condition': 128,  # Dimensionality of conditioning vectors (default: 128)
    'n_condition': 7,  # Number of distinct condition properties (default: 7)
})

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:

# preprocess train data, validation data and test data. Only once for the first time that you run the code. Then the appropriate .pt files will be saved and loaded.
trainset = preprocess_dataset("train", args.n_max_nodes, args.spectral_emb_dim)
validset = preprocess_dataset("valid", args.n_max_nodes, args.spectral_emb_dim)
testset = preprocess_dataset("test", args.n_max_nodes, args.spectral_emb_dim)


# initialize data loaders
train_loader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=args.batch_size, shuffle=False)
test_loader = DataLoader(testset, batch_size=args.batch_size, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt
def plot_graph(data):
    # Assuming `data` is a single graph from the trainset
    edge_index = data.edge_index.cpu().numpy()
    node_features = data.x.cpu().numpy()

    # Create a NetworkX graph
    G = nx.Graph()

    # Add nodes with features
    num_nodes = node_features.shape[0]
    for i in range(num_nodes):
        G.add_node(i, feature=node_features[i])

    # Add edges
    for edge in edge_index.T:  # Transpose to get pairs of edges
        G.add_edge(edge[0], edge[1])

    # Plot the graph
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G)  # You can choose other layouts like `circular_layout`, etc.
    nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', font_size=12, font_weight='bold', edge_color='gray')
    plt.title("Graph Visualization")
    plt.show()

# Example: Plot the first graph in the trainset
graph_data = trainset[0]  # Assuming trainset is a list of graphs
plot_graph(graph_data)

In [ ]:
graph_data = trainset[100]  # Assuming trainset is a list of graphs
plot_graph(graph_data)

In [ ]:
import matplotlib.pyplot as plt

# Initialize lists to store losses
train_losses = []
val_losses = []
train_recon_losses = []
val_recon_losses = []
train_kld_losses = []
val_kld_losses = []

# Initialize VGAE model and optimizer
autoencoder = VariationalAutoEncoder(args.spectral_emb_dim + 1,
                                     args.hidden_dim_encoder,
                                     args.hidden_dim_decoder,
                                     args.latent_dim,
                                     args.n_layers_encoder,
                                     args.n_layers_decoder,
                                     args.n_max_nodes).to(device)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.1)

# Train VGAE model
if args.train_autoencoder:
    best_val_loss = np.inf
    for epoch in range(1, args.epochs_autoencoder + 1):
        autoencoder.train()
        train_loss_all = 0
        train_count = 0
        train_loss_all_recon = 0
        train_loss_all_kld = 0
        cnt_train = 0

        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()

            # Check for NaN in the relevant tensors inside the data object
            if torch.isnan(data.x).any() or torch.isnan(data.edge_index).any():
                print("NaN detected in data!")
                continue

            loss, recon, kld = autoencoder.loss_function(data)

            # Check for NaN in loss
            if torch.isnan(loss).any() or torch.isnan(recon).any() or torch.isnan(kld).any():
                print(f"NaN detected in loss (Epoch {epoch})")
                continue

            train_loss_all_recon += recon.item()
            train_loss_all_kld += kld.item()
            cnt_train += 1
            loss.backward()

            # Gradient clipping to avoid explosion
            torch.nn.utils.clip_grad_norm_(autoencoder.parameters(), max_norm=1.0)

            train_loss_all += loss.item()
            train_count += torch.max(data.batch) + 1
            optimizer.step()

        autoencoder.eval()
        val_loss_all = 0
        val_count = 0
        cnt_val = 0
        val_loss_all_recon = 0
        val_loss_all_kld = 0

        for data in val_loader:
            data = data.to(device)
            loss, recon, kld = autoencoder.loss_function(data)
            val_loss_all_recon += recon.item()
            val_loss_all_kld += kld.item()
            val_loss_all += loss.item()
            cnt_val += 1
            val_count += torch.max(data.batch) + 1

        # Store the losses for plotting
        train_losses.append(train_loss_all / cnt_train)
        val_losses.append(val_loss_all / cnt_val)
        train_recon_losses.append(train_loss_all_recon / cnt_train)
        val_recon_losses.append(val_loss_all_recon / cnt_val)
        train_kld_losses.append(train_loss_all_kld / cnt_train)
        val_kld_losses.append(val_loss_all_kld / cnt_val)

        if epoch % 10 == 0:
            dt_t = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
            print(f'{dt_t}',
                  f'Epoch: {epoch:04d}',
                  f'T_l: {train_loss_all/cnt_train:.5f}',
                  f'T_rec_l: {train_loss_all_recon/cnt_train:.2f}',
                  f'T_KLD_l: {train_loss_all_kld/cnt_train:.2f}',
                  f'V_l: {val_loss_all/cnt_val:.5f}',
                  f'V_rec_l: {val_loss_all_recon/cnt_val:.2f}',
                  f'V_KLD_l: {val_loss_all_kld/cnt_val:.2f}')

        scheduler.step()

        if best_val_loss >= val_loss_all:
            best_val_loss = val_loss_all
            torch.save({
                'state_dict': autoencoder.state_dict(),
                'optimizer' : optimizer.state_dict(),
            }, 'autoencoder.pth.tar')

    # After training is complete, plot and save the loss curves
    # Plot total loss
    plt.figure()
    plt.plot(range(1, args.epochs_autoencoder + 1), train_losses, label='Train Loss')
    plt.plot(range(1, args.epochs_autoencoder + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train vs Validation Loss')
    plt.legend()
    plt.savefig('loss_plot.png')  # Save plot as a PNG file

    # Plot reconstruction loss
    plt.figure()
    plt.plot(range(1, args.epochs_autoencoder + 1), train_recon_losses, label='Train Reconstruction Loss')
    plt.plot(range(1, args.epochs_autoencoder + 1), val_recon_losses, label='Validation Reconstruction Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Reconstruction Loss')
    plt.title('Train vs Validation Reconstruction Loss')
    plt.legend()
    plt.savefig('recon_loss_plot.png')  # Save plot as a PNG file

    # Plot KLD loss
    plt.figure()
    plt.plot(range(1, args.epochs_autoencoder + 1), train_kld_losses, label='Train KLD Loss')
    plt.plot(range(1, args.epochs_autoencoder + 1), val_kld_losses, label='Validation KLD Loss')
    plt.xlabel('Epoch')
    plt.ylabel('KLD Loss')
    plt.title('Train vs Validation KLD Loss')
    plt.legend()
    plt.savefig('kld_loss_plot.png')  # Save plot as a PNG file

else:
    checkpoint = torch.load('autoencoder.pth.tar')
    autoencoder.load_state_dict(checkpoint['state_dict'])

autoencoder.eval()

# define beta schedule
betas = linear_beta_schedule(timesteps=args.timesteps)

# define alphas
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)

# calculations for diffusion q(x_t | x_{t-1}) and others
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)

# calculations for posterior q(x_{t-1} | x_t, x_0)
posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

# initialize denoising model
denoise_model = DenoiseNN(input_dim=args.latent_dim, hidden_dim=args.hidden_dim_denoise, n_layers=args.n_layers_denoise, n_cond=args.n_condition, d_cond=args.dim_condition).to(device)
# de-hashtag if two GPU are accesible
# denoise_model = nn.DataParallel(denoise_model, device_ids=[0, 1])

optimizer = torch.optim.Adam(denoise_model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.1)

# Initialize loss tracking lists
train_losses_denoiser = []
val_losses_denoiser = []

if args.train_denoiser:
    best_val_loss = np.inf
    for epoch in range(1, args.epochs_denoise + 1):
        denoise_model.train()
        train_loss_all = 0
        train_count = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()

            if torch.isnan(data.x).any() or torch.isnan(data.edge_index).any():
                print("NaN detected in data!")

            x_g = autoencoder.encode(data)
            t = torch.randint(0, args.timesteps, (x_g.size(0),), device=device).long()
            loss = p_losses(denoise_model, x_g, t, data.stats, sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, loss_type="huber")

            if torch.isnan(loss).any():
                print(f"NaN detected in loss (Epoch {epoch})")
                continue

            loss.backward()

            torch.nn.utils.clip_grad_norm_(denoise_model.parameters(), max_norm=1.0)

            train_loss_all += x_g.size(0) * loss.item()
            train_count += x_g.size(0)
            optimizer.step()

        # Validation loss
        denoise_model.eval()
        val_loss_all = 0
        val_count = 0
        for data in val_loader:
            data = data.to(device)
            x_g = autoencoder.encode(data)
            t = torch.randint(0, args.timesteps, (x_g.size(0),), device=device).long()
            loss = p_losses(denoise_model, x_g, t, data.stats, sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, loss_type="huber")
            val_loss_all += x_g.size(0) * loss.item()
            val_count += x_g.size(0)

        # Store losses for plotting
        train_losses_denoiser.append(train_loss_all / train_count)
        val_losses_denoiser.append(val_loss_all / val_count)

        if epoch % 5 == 0:
            dt_t = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
            print(f'{dt_t}, Epoch: {epoch:04d}, Train Loss: {train_loss_all/train_count:.5f}, Val Loss: {val_loss_all/val_count:.5f}')

        scheduler.step()

        if best_val_loss >= val_loss_all:
            best_val_loss = val_loss_all
            torch.save({
                'state_dict': denoise_model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }, 'denoise_model.pth.tar')

    # Plotting the losses
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, args.epochs_denoise + 1), train_losses_denoiser, label='Train Loss')
    plt.plot(range(1, args.epochs_denoise + 1), val_losses_denoiser, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Denoiser Training and Validation Loss')
    plt.legend()
    plt.savefig('denoiser_loss_plot.png')  # Save the plot
    plt.show()  # Show the plot

else:
    checkpoint = torch.load('denoise_model.pth.tar')
    denoise_model.load_state_dict(checkpoint['state_dict'])

denoise_model.eval()



# Save to a CSV file
with open("output.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["graph_id", "edge_list"])
    for data in tqdm(test_loader, desc='Processing test set'):
        data = data.to(device)
        graph_ids, stats = data.filename, data.stats
        samples = sample(denoise_model, stats, latent_dim=args.latent_dim, timesteps=args.timesteps, betas=betas, batch_size=stats.size(0))
        adj = autoencoder.decode_mu(samples[-1],stats)

        for i, graph_id in enumerate(graph_ids):
            stat_x = stats[i].view(-1, args.n_condition).detach().cpu().numpy()
            Gs_generated = construct_nx_from_adj(adj[i].detach().cpu().numpy())
            edge_list_text = ", ".join([f"({u}, {v})" for u, v in Gs_generated.edges()])
            writer.writerow([graph_id, edge_list_text])

In [ ]:
# Save to a CSV file
with open("output.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["graph_id", "edge_list"])
    for data in tqdm(test_loader, desc='Processing test set'):
        data = data.to(device)
        graph_ids, stats = data.filename, data.stats
        samples = sample(denoise_model, stats, latent_dim=args.latent_dim, timesteps=args.timesteps, betas=betas, batch_size=stats.size(0))
        adj = autoencoder.decode_mu(samples[-1],data.stats)
        for i, graph_id in enumerate(graph_ids):
            stat_x = stats[i].view(-1, args.n_condition).detach().cpu().numpy()
            Gs_generated = construct_nx_from_adj(adj[i].detach().cpu().numpy())
            edge_list_text = ", ".join([f"({u}, {v})" for u, v in Gs_generated.edges()])
            writer.writerow([graph_id, edge_list_text])

In [ ]:
import numpy as np
from tqdm import tqdm

# Function to normalize features
def normalize_features(features):
    """
    Normalize a set of features using min-max normalization.

    Parameters:
        features (numpy.ndarray): Array of features to normalize.

    Returns:
        numpy.ndarray: Normalized features.
        tuple: Min and max values for each feature (for future scaling if needed).
    """
    min_vals = np.min(features, axis=0)
    max_vals = np.max(features, axis=0)
    range_vals = max_vals - min_vals

    # Avoid division by zero
    range_vals[range_vals == 0] = 1

    normalized_features = (features - min_vals) / range_vals
    return normalized_features, min_vals, max_vals


# Compute MAE for the test set
np.set_printoptions(suppress=True, precision=2)

MAE_test = 0
all_gt_stats = []  # Collect all ground truth stats
all_pred_stats = []  # Collect all predicted stats

for data in tqdm(test_loader, desc='Processing test set'):
    data = data.to(device)
    graph_ids, stats = data.filename, data.stats
    samples = sample(denoise_model, stats, latent_dim=args.latent_dim, timesteps=args.timesteps, betas=betas, batch_size=stats.size(0))
    adj = autoencoder.decode_mu(samples[-1],data.stats)

    for i, graph_id in enumerate(graph_ids):
        graph_features_pred = graph_features(adj[i].detach().cpu().numpy())
        if i % 50 == 0:
            print('Stats GT:', np.round(stats[i].detach().cpu().numpy(), 2))
            print('Stats PR:', np.round(graph_features_pred, 2))

        # Collect ground truth and predicted stats
        all_gt_stats.append(stats[i].detach().cpu().numpy())
        all_pred_stats.append(graph_features_pred)

# Convert collected stats to numpy arrays
all_gt_stats = np.array(all_gt_stats)
all_pred_stats = np.array(all_pred_stats)

# Normalize ground truth and predicted stats
normalized_gt_stats, min_vals, max_vals = normalize_features(all_gt_stats)
normalized_pred_stats = (all_pred_stats - min_vals) / (max_vals - min_vals)
normalized_pred_stats[np.isnan(normalized_pred_stats)] = 0  # Handle potential NaNs

# Compute normalized MAE
MAE_test = np.mean(np.abs(normalized_gt_stats - normalized_pred_stats))
print('Normalized MAE test set:', MAE_test)

In [ ]:
n_samples = 30
def compute_mae(adj, graph_ids, stats):
    features_pred_batch = []
    features_gt_batch = []

    for i, graph_id in enumerate(graph_ids):
        graph_features_pred = graph_features(adj[i].detach().cpu().numpy())
        stats_gt = stats[i].detach().cpu().numpy()
        features_pred_batch.append(graph_features_pred)
        features_gt_batch.append(stats_gt)

    features_pred_batch = np.array(features_pred_batch)
    features_gt_batch = np.array(features_gt_batch)

    # Normalize ground truth features
    norm_gt_stats_batch, min_vals, max_vals = normalize_features(features_gt_batch)

    # Normalize predicted features
    norm_pred_stats_batch = (features_pred_batch - min_vals) / (max_vals - min_vals)
    norm_pred_stats_batch[np.isnan(norm_pred_stats_batch)] = 0  # Handle potential NaNs

    # Calculate Mean Absolute Error
    MAE = np.mean(np.abs(norm_gt_stats_batch - norm_pred_stats_batch))

    return MAE
def normalize_features(features):
    """
     Normalize a set of features using min-max normalization.

    Parameters:
        features (numpy.ndarray): Array of features to normalize.

    Returns:
        numpy.ndarray: Normalized features.
        tuple: Min and max values for each feature (for future scaling if needed).
    """
    min_vals = np.min(features, axis=0)
    max_vals = np.max(features, axis=0)
    range_vals = max_vals - min_vals

    # Avoid division by zero
    range_vals[range_vals == 0] = 1

    normalized_features = (features - min_vals) / range_vals
    return normalized_features, min_vals, max_vals

with open("output.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["graph_id", "edge_list"])
    for data in tqdm(test_loader, desc='Processing test set'):
        data = data.to(device)
        graph_ids, stats = data.filename, data.stats
        best_MAE = 1
        for k in range(n_samples):
            samples = sample(denoise_model, stats, latent_dim=args.latent_dim, timesteps=args.timesteps, betas=betas, batch_size=stats.size(0))
            adj = autoencoder.decode_mu(samples[-1], stats)
            mae = compute_mae(adj, graph_ids, stats)
            if mae < best_MAE:
                best_MAE = mae
                best_samples = samples
        adj = autoencoder.decode_mu(best_samples[-1], stats)
        for i, graph_id in enumerate(graph_ids):
            stat_x = stats[i].view(-1, args.n_condition).detach().cpu().numpy()
            Gs_generated = construct_nx_from_adj(adj[i].detach().cpu().numpy())
            edge_list_text = ", ".join([f"({u}, {v})" for u, v in Gs_generated.edges()])
            writer.writerow([graph_id, edge_list_text])

In [ ]:
# Compute MAE for validation set
MAE_val = 0
for data in tqdm(val_loader, desc='Processing validation set'):
    data = data.to(device)
    graph_ids, stats = data.filename, data.stats
    samples = sample(denoise_model, stats, latent_dim=args.latent_dim, timesteps=args.timesteps, betas=betas, batch_size=stats.size(0))
    adj = autoencoder.decode_mu(samples[-1])

    for i, graph_id in enumerate(graph_ids):
        Gs_generated = construct_nx_from_adj(adj[i].detach().cpu().numpy())

        if i % 500 == 0 :
            print('stats gt :', np.round(stats[i].detach().cpu().numpy(), 2))
            print('stats pr :', np.round(graph_features(Gs_generated.edges()), 2))

        MAE_val += np.sum(np.abs(stat_x - graph_features(Gs_generated.edges())))

MAE_val /= len(val_loader.dataset)
print('MAE validation set:', MAE_val)




In [ ]:
# Compute MAE for training set
MAE_train = 0
for data in tqdm(train_loader, desc='Processing training set'):
    data = data.to(device)
    graph_ids, stats = data.filename, data.stats
    samples = sample(denoise_model, stats, latent_dim=args.latent_dim, timesteps=args.timesteps, betas=betas, batch_size=stats.size(0))
    adj = autoencoder.decode_mu(samples[-1])

    for i, graph_id in enumerate(graph_ids):
        Gs_generated = construct_nx_from_adj(adj[i].detach().cpu().numpy())
        if i % 4000 == 0 :
            print('stats gt :', np.round(stats[i].detach().cpu().numpy(), 2))
            print('stats pr :', np.round(graph_features(Gs_generated.edges()), 2))

        MAE_train += np.sum(np.abs(stat_x - graph_features(Gs_generated.edges())))
MAE_train /= len(train_loader.dataset)
print('MAE train set:', MAE_train)